# Ejemplo de extracción de datos

Este notebook utiliza una colección digital descrita a través de ficheros MARCXML que incluye metadatos descriptivos del catálogo [Moving Image Archive](https://data.nls.uk/data/metadata-collections/moving-image-archive/) de la Biblioteca Nacional de Escocia.

### Importando las librerías de código

In [1]:
# https://pypi.org/project/pymarc/
import pymarc, re, csv
import pandas as pd
from pymarc import parse_xml_to_array

### Generando un fichero CSV como salida con el contenido procesado

In [2]:
csv_salida = csv.writer(open('registros_marc.csv', 'w'), delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
csv_salida.writerow(['titulo', 'autor', 'lugar_produccion', 'fecha', 'extension', 'creditos', 'materias', 'resumen', 'detalles', 'enlace'])

89

### Generando un fichero CSV como salida con el contenido procesado a partir de los archivos originales

In [10]:
with open('registros_marc.csv', 'w') as csv_sal:
    csv_salida = csv.writer(csv_sal, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    csv_salida.writerow(['titulo', 'autor', 'lugar_produccion', 'fecha', 'extension', 'creditos', 'materias', 'resumen', 'detalles', 'enlace'])


    registros = parse_xml_to_array(open('Moving-Image-Archive/Moving-Image-Archive-dataset-MARC.xml'))

    for registro in registros:

        titulo = autor = lugar_produccion = fecha = extension = creditos = materias = resumen = detalles = enlace =''

        # titulo
        if registro['245'] is not None:
          titulo = registro['245']['a']
          if registro['245']['b'] is not None:
            titulo = titulo + " " + registro['245']['b']

        # autor
        if registro['100'] is not None:
          autor = registro['100']['a']
        elif registro['110'] is not None:
          autor = registro['110']['a']
        elif registro['700'] is not None:
          autor = registro['700']['a']
        elif registro['710'] is not None:
          autor = registro['710']['a']

        # lugar de producción
        if registro['264'] is not None:
          lugar_produccion = registro['264']['a']

        # fecha
        for f in registro.get_fields('264'):
            fechas = f.get_subfields('c')
            if len(fechas):
                fecha = fechas[0]

                if fecha.endswith('.'): fecha = fecha[:-1]


        # Physical Description - extent
        for f in registro.get_fields('300'):
            extension = f.get_subfields('a')
            if len(extension):
                extension = extension[0]
                # TODO cleaning
            detalles = f.get_subfields('b')
            if len(detalles):
                detalles = detalles[0]

        # creditos
        if registro['508'] is not None:
          creditos = registro['508']['a']

        # Resumen
        if registro['520'] is not None:
          resumen = registro['520']['a']

        # Materia
        if registro['653'] is not None:
            materias = '' 
            for f in registro.get_fields('653'):
                materias += f.get_subfields('a')[0] + ' -- '
            materias = re.sub(' -- $', '', materias)


        # enlace
        if registro['856'] is not None:
          enlace = registro['856']['u']


        csv_salida.writerow([titulo,autor,lugar_produccion,fecha,extension,creditos,materias,resumen,detalles,enlace])

## Leyendo el fichero CSV 

In [11]:
# Este comando añade el contenido del fichero a un Pandas DataFrame
df = pd.read_csv('registros_marc.csv')

## Consultando el contenido

In [12]:
df

,titulo,autor,lugar_produccion,fecha,extension,creditos,materias,resumen,detalles,enlace
0,(GLASGOW TRAMS AND BOTANIC GARDENS).,"RUSSELL, Stanley Livingstone",[Place of production not identified] :,1950.0,(2.00 mins) :,"Director, [filmed by Stanley L. Russell, Thame...","Bus Stations and Depots -- Buses and Coaches, ...","The Botanic Gardens, Glasgow with shots of the...","mute, colour",http://movingimage.nls.uk/film/0001
1,"(LAST DAY OF THE TRAMS, GLASGOW).",NaN,[Place of production not identified] :,1962.0,(28.00 mins) :,"Director, [filmed by SAAC].",Transport -- Glasgow -- documentary -- amateur,"Footage of the last trams to run in Glasgow, a...","silent, colour",http://movingimage.nls.uk/film/0002
2,INTO THE MISTS.,NaN,[Place of production not identified] :,1956.0,(10.04 mins) :,"Director, [filmed by W.S. Dobson].","Ceremonies -- Emotions, Attitudes and Behaviou...",The story of the last Edinburgh tram. Shots o...,"silent, colour",http://movingimage.nls.uk/film/0004
3,"PASSING OF THE TRAMCAR, the.",NaN,[Place of production not identified] :,1962.0,(63.36 mins) :,NaN,Ceremonies -- Transport -- Glasgow,Footage of the last tram to run in Glasgow. Th...,"silent, colour",http://movingimage.nls.uk/film/0005
4,SCOTS OF TOMORROW.,Campbell Harper Productions,[Place of production not identified] :,1959.0,(13.00 mins) :,"Producer, Campbell Harper Films Ltd..","Art and Artists, general -- Education -- edu...",Scottish school pupils studying scientific and...,"sound, black and white",http://movingimage.nls.uk/film/0007
...,...,...,...,...,...,...,...,...,...,...
6012,CITY OF BIRMINGHAM .,NaN,[Place of production not identified] :,1948.0,(6.11 mins) :,NaN,Ceremonies -- Construction and Engineering -- ...,Built and engined by John Brown & Co. Ltd. S...,"silent, colour",http://movingimage.nls.uk/film/UCS0195
6013,BUILDING THE BIG DREDGE - STAGE 1.,NaN,[Place of production not identified] :,1964.0,(8min20sec) :,"Producer, Stephen Group Film Unit.",Construction and Engineering -- Ships and Ship...,"Shots of Indonesian Sea Dredge No. 1, under co...","silent, colour",http://movingimage.nls.uk/film/UCS0204
6014,ALEXANDER STEPHEN'S YARD.,NaN,[Place of production not identified] :,1964.0,(11.57 mins) :,"Producer, .","Employment, Industry and Industrial Relations ...","Shots of the Alexander Stephen's yard, and the...","silent, colour",http://movingimage.nls.uk/film/UCS0207
6015,QUEEN ELIZABETH Ship No. 552.,NaN,[Place of production not identified] :,1940.0,(5min24sec) :,NaN,"Employment, Industry and Industrial Relations ...",Built and engineered by John Brown & Co. Ltd. ...,"silent, black and white",http://movingimage.nls.uk/film/UCS0213


### Consultando las columnas

In [13]:
df.columns

Index(['titulo', 'autor', 'lugar_produccion', 'fecha', 'extension', 'creditos',
       'materias', 'resumen', 'detalles', 'enlace'],
      dtype='object')

### ¿Cuántos registros existen?

In [14]:
len(df)

6017

## Explorando las materias
### Creamos una lista de materias y la ordenamos alfabéticamente

In [15]:
df['materias'][2]

'Ceremonies -- Emotions, Attitudes and Behaviour -- Local Government -- Transport -- Edinburgh -- amateur'

In [16]:
df['materias'].str.split('--', expand=True).stack()

0     0                  Bus Stations and Depots 
      1               Buses and Coaches, general 
      2     Celebrations, Traditions and Customs 
      3                     Children and Infants 
      4                   Leisure and Recreation 
                            ...                  
6016  0                                Carriages 
      1                               Ceremonies 
      2                       Ships and Shipping 
      3                           Dunbartonshire 
      4                                 technical
Length: 23742, dtype: object

In [17]:
# Obtener valores únicos
materias = pd.unique(df['materias'].str.split(' -- ', expand=True).stack()).tolist()
for materia in sorted(materias, key=str.lower):
    print(materia)

Aberdeen
Aberdeenshire
advertising
Agriculture
Air displays and shows
Air Raids
Aircraft see also Helicopters
Airports
amateur
Angus
Animals
animation
Architecture and Buildings
Argyllshire
Art and Artists, general  
Arts and Crafts
Ayrshire
Banff
Berwickshire
biographical
Birds
Borders
British Empire, the
Broadcasting, general
Buddhism
Bulldozers
Bus Stations and Depots
Buses and Coaches, general
Butchers and Butcher Shops
Bute
Cafeterias and Canteens
Caithness
Camping
Canals
Canoeing
Carriages
Celebrations, Traditions and Customs
Celts and Celtic Culture
Ceremonies
Cheese and Cheese Making
Children and Infants
children's
Christmas  see also New Year
cine mag
Clackmannanshire
comedy
Construction and Engineering
crime
Crime, Punishment and Law Enforcement
dance
Dentistry
Depression, the
Disillusionment
documentary
Dumfriesshire
Dunbartonshire
Dundee
East Lothian
Easter
Edinburgh
Education
educational
Emotions, Attitudes and Behaviour
Employment, Industry and Industrial Relations
Enviro

In [18]:
df['materias']

0       Bus Stations and Depots -- Buses and Coaches, ...
1          Transport -- Glasgow -- documentary -- amateur
2       Ceremonies -- Emotions, Attitudes and Behaviou...
3                      Ceremonies -- Transport -- Glasgow
4       Art and Artists, general   -- Education -- edu...
                              ...                        
6012    Ceremonies -- Construction and Engineering -- ...
6013    Construction and Engineering -- Ships and Ship...
6014    Employment, Industry and Industrial Relations ...
6015    Employment, Industry and Industrial Relations ...
6016    Carriages -- Ceremonies -- Ships and Shipping ...
Name: materias, Length: 6017, dtype: object

### También podemos calcular con qué frecuencia se usa una materia

In [19]:
# Partir las materias y obtener el número de ocurrencias
materia_contador = df['materias'].str.split(' -- ').apply(lambda x: pd.Series(x).value_counts()).sum().astype('int').sort_values(ascending=False).to_frame().reset_index(level=0)
# Añadimos las columnas
materia_contador.columns = ['materia', 'contador']
# Mostrar con barras horizontales
display(materia_contador.style.bar(subset=['contador'], color='#d65f5f').set_properties(subset=['contador'], **{'width': '300px'}))

,materia,contador
0,amateur,2023
1,Leisure and Recreation,813
2,Glasgow,797
3,documentary,707
4,Transport,674
5,"Employment, Industry and Industrial Relations",632
6,television news,542
7,Edinburgh,538
8,Sporting Activities,525
9,"Celebrations, Traditions and Customs",453
